In [1]:
print('hello')

hello


In [16]:
import torch

x = torch.zeros(4, 8)
x.shape, x.dtype == torch.float32

(torch.Size([4, 8]), True)

In [17]:
x.numel()

32

In [18]:
x.element_size() == 4 # 4 bytes

True

In [19]:
x.element_size()

4

In [21]:
def get_memory_usage(x):
    return x.numel() * x.element_size()

get_memory_usage(x) == 4 * 8 * 4

True

In [29]:
# one matrix in the feed forward layer in gpt-3 memory usage
get_memory_usage(torch.empty(12288 * 4, 12288)) / 1024 / 1024 / 1024  # 2.3GB

2.25

In [34]:
x = torch.zeros((4, 8), dtype=torch.float16)
x.shape, x.element_size()

(torch.Size([4, 8]), 2)

In [37]:
get_memory_usage(torch.empty((12288 * 4, 12288),  dtype=torch.float16)) / 1024 / 1024 / 1024  # 1.12 GB

1.125

In [39]:
x = torch.tensor([1e-8])
x

tensor([1.0000e-08])

In [41]:
x = torch.tensor([1e-8], dtype=torch.float16)
x

tensor([0.], dtype=torch.float16)

In [42]:
x = torch.tensor([1e-8], dtype=torch.bfloat16)
x

tensor([1.0012e-08], dtype=torch.bfloat16)

In [43]:
get_memory_usage(torch.empty((12288 * 4, 12288),  dtype=torch.bfloat16)) / 1024 / 1024 / 1024  # 1.12 GB

1.125

In [46]:
x = torch.zeros(32, 32)
x.device == torch.device('cpu')

True

In [47]:
torch.cuda.is_available()

False

In [ ]:
x = torch.tensor([
    [0., 1, 2, 3],
    [4, 5, 6, 7],
    [8, 9, 10, 11],
    [12, 13, 14, 15]
])

In [ ]:
x.stride(0), x.stride(1)

(4, 1, <function Tensor.stride>)

In [54]:
r, c = 1, 2
index = r * x.stride(0) + c * x.stride(1)
index

6

In [56]:
x = torch.tensor([
    [1, 2, 3],
    [4, 5, 6]
])
y = x[0]

In [60]:
def same_storage(x, y):
    return x.untyped_storage().data_ptr() == y.untyped_storage().data_ptr()

In [61]:
same_storage(x, y)

True

In [62]:
x.untyped_storage().data_ptr()

140453203478976

In [65]:
y = x.view(3, 2)
y.shape, x.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [66]:
same_storage(x, y)

True

In [68]:
same_storage(x, y.view(3,2).contiguous())

True

In [71]:
import torch


B = 16384
D = 32768
K = 8192

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.memory_allocated()

0

In [73]:
x = torch.ones(B, D, device=device)
w = torch.randn(D, K, device=device)
y = x @ w
# we have one multiplication x[i][j] * w[j][k] and one addition per i,j,k triplets
num_flops = 2 * B * D * K
num_flops

8796093022208